In [2]:
import tensorflow as tf
import os
import random
import math
import sys
from PIL import Image
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
# 验证集数量
_NUM_TEST = 500
# 随机数种子
_RANDOM_SEED = 0
# 数据集文件夹
_DATA_DIR = "c:/learn/captcha/images"
# TFrecord文件夹
TFRECORD_DIR = "c:/learn/captcha"

# 获取所有验证码图片
def _get_filenames_and_classes(data_dir):
    photo_filenames = []
    for filename in os.listdir(data_dir):
        # 获取文件路径
        path = os.path.join(data_dir, filename)
        photo_filenames.append(path)
    return photo_filenames

def int64_frature(values):
    if not isinstance(values, (tuple, list)):
        values = [values]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=values))

def bytes_feature(values):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[values]))

def image_to_tfexample(image_data, label0, label1, label2, label3):
    
    return tf.train.Example(features=tf.train.Features(feature={
        "image"  : bytes_feature(image_data), 
        "label0" : int64_frature(label0), 
        "label1" : int64_frature(label1), 
        "label2" : int64_frature(label2), 
        "label3" : int64_frature(label3),
    }))

def _convert_dataset(split_name, filenames, dataset_dir):
    assert split_name in ["train", "test"]
    
    with tf.Session() as sess:
        # 定义tfrecord文件的路径和名字
        output_filename = os.path.join(TFRECORD_DIR, split_name + ".tfrecords")
        with tf.python_io.TFRecordWriter(output_filename) as tfrecord_writer:
            
            for i, filename in enumerate(filenames):
                
                try:
                    print("正在转换第 %d 张图片,共 %d 张" %(i+1, len(filenames)),end="\r")
                    
                    # 读取图片
                    image_data = Image.open(filename)
                    # 根据模型结构resize
                    image_data.resize((224, 224))
                    # 灰度化
                    image_data = np.array(image_data.convert("L"))
                    # 将图片转化成bytes
                    image_data = image_data.tobytes()
                    
                    # 获取label
                    labels = filename.split("\\")[-1][0:4]
                    num_labels = []
                    for j in range(4):
                        num_labels.append(int(labels[j]))
                    
                    # 生成protocl数据类型
                    example = image_to_tfexample(image_data, 
                                                 num_labels[0], 
                                                num_labels[1], 
                                                num_labels[2], 
                                                num_labels[3])
                    tfrecord_writer.write(example.SerializeToString())
                except IOError as e:
                    print("Could not read ", filename)
                    print("Error:", e)
                    print("Skip it")

# 获取所有图片
photo_filenames = _get_filenames_and_classes(_DATA_DIR)

# 把数据切分成训练集和测试机，并且打乱
random.seed(_RANDOM_SEED)
random.shuffle(photo_filenames)
trainning_filenames = photo_filenames[_NUM_TEST:]
testing_filenames = photo_filenames[:_NUM_TEST]

# 数据转换
_convert_dataset("train", trainning_filenames, _DATA_DIR)
_convert_dataset("test", testing_filenames, _DATA_DIR)

print("tfrecord文件已经生成")

tfrecord文件已经生成共 500 张 张
